This Notebook shows the example usage of estimating the electron lifetime of a purification dataset using the LifetimeEstimation class. It starts with the loading of reduced data from a run, then it illustrates the steps needed to be taken by the user to use the function to their advantage. In summary, the user must apply quality cuts to the data that make the alpha band visible, then pass these cuts along with a reasonable guess of the lifetime and intercept of the alpha band energy vs drift time into the class, wherein it estimates the lifetime.

Start with importing necessary modules and setting up plot parameters

In [8]:
import sys
import cycler
import numpy as np
import matplotlib.pyplot as plt
from StanfordTPCAnalysis.StruckAnalysisConfiguration import StruckAnalysisConfiguration
import time
import pickle
from StanfordTPCAnalysis.LifetimeEstimation import LifetimeEstimation as LE

# Augment the path environment variable
sys.path.append('/g/g20/lenardo1/software')

# Set up Plot Stuff
plt.rcParams['axes.prop_cycle'] = cycler.cycler(color='bgrmyk')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

ModuleNotFoundError: No module named 'StanfordTPCAnalysis'

Load in an example set; DS05 reduced v9 from Run 34 contains one of the least accurate fits of the datasets on which this class was tested

In [ ]:
# Run 34 DS05 reduced_v9
path_to_reduced_data = '/p/lustre2/nexouser/hardy27/StanfordTPCData/purification_data/'
run_num = 'Run34'
this_dataset = 'DS05'  
reduced_num = '/reduced_v9/' # reduced_v4 for DS07
fname = path_to_reduced_data + run_num + '/' + this_dataset + reduced_num + 'reduced_added.p'  

#all the run configuration paramenters are stored in these three files
config_path = '/config/Run34_v2/'
run_parameters_file = path_to_reduced_data + run_num + '/' + this_dataset + config_path + 'Run_Parameters.csv'
calibrations_file = path_to_reduced_data + run_num + '/' + this_dataset + config_path + 'Calibrations_Xe.csv'
channel_map_file = path_to_reduced_data + run_num + '/' + this_dataset + config_path + 'Channel_Map.csv'

#analysis_config object loads all these paramenters
analysis_config = StruckAnalysisConfiguration.StruckAnalysisConfiguration()

analysis_config.GetRunParametersFromFile( run_parameters_file, sheet = this_dataset )
analysis_config.GetChannelMapFromFile( channel_map_file, sheet = this_dataset )

with open(fname,'rb') as input_file:
    data_df = pickle.load(input_file)

Plot the uncut data first

In [ ]:
# Tile, Drift Time Data
charge_energy = data_df['TotalTileEnergy']
sipm_energy = data_df['TotalSiPMEnergy']
time_of_max_channel = data_df['TimeOfMaxChannel']
drift_time = (time_of_max_channel - analysis_config.run_parameters['Pretrigger Length [samples]']) \
               * analysis_config.run_parameters['Sampling Period [ns]'] / 1000.

# Plot uncut Tile Energy, SiPM Energy vs. Drift Time
plt.figure(0)
plt.plot(drift_time,charge_energy,'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.))
plt.xlabel("Drift time (µs)")
plt.ylabel("Charge Tile Energy (ADC Counts)")
plt.title("Tile Energy vs. Drift Time, " + run_num + ", " + this_dataset)

plt.figure(1)
plt.plot(drift_time,sipm_energy,'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.))
plt.xlabel("Drift time (µs)")
plt.ylabel("SiPM Energy (ADC Counts)")
plt.title("SiPM Energy vs. Drift Time, " + run_num + ", " + this_dataset)

Then apply quality cuts until the alpha band is clear

In [ ]:
# Preliminary Quality Cuts
start_drift = 10 # µs 
end_drift = 45 # µs 
drift_interval = 5 # µs 
sipm_lower_bound = 20000 # ADC Counts
sipm_upper_bound = 1000000 # ADC Counts
charge_lower_bound = 250  # ADC Counts 
charge_upper_bound = 750  # ADC Counts 
num_tile_channels_hit = 3 # Channels

Visualize the data until the alpha band is clear

In [ ]:
# Apply Cuts to Data to Visualize the Alpha Band; from this, guess the lifetime and intercept
mask = (data_df['TotalTileEnergy']>charge_lower_bound) & (data_df['TotalTileEnergy']<charge_upper_bound) \
        & (data_df['TotalSiPMEnergy']>sipm_lower_bound) & (data_df['TotalSiPMEnergy']<sipm_upper_bound) \
        & (data_df['NumTileChannelsHit'] < num_tile_channels_hit) & (data_df['IsFull3D'])

plt.figure(0)
plt.plot(drift_time[mask],charge_energy[mask],'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.))
plt.xlabel("Drift time (µs)")
plt.ylabel("Charge Tile Energy (ADC Counts)")
plt.title("Tile Energy vs. Drift Time, " + run_num + ", " + this_dataset)

plt.figure(1)
plt.plot(drift_time[mask],sipm_energy[mask],'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.))
plt.xlabel("Drift time (µs)")
plt.ylabel("SiPM Energy (ADC Counts)")
plt.title("SiPM Energy vs. Drift Time, " + run_num + ", " + this_dataset)

Make a guess of the lifetime and intercept

In [ ]:
intercept_guess = 650 # ADC Counts
lifetime_guess = 110 # µs 

Pass these guesses and your quality cuts that made the alpha band clear into the class function for a lifetime estimate

In [ ]:
# Estimate the Lifetime
lifetime, sigma_lifetime, intercept, sigma_intercept, r_squared,\
    drift_slices, centroids, centroids_stds = LE.binnedElectronLifetimeFit(intercept_guess,\
                                                                           lifetime_guess,start_drift,end_drift,\
                                                                           data_df, analysis_config,sipm_lower_bound,\
                                                                           sipm_upper_bound,charge_lower_bound,\
                                                                           charge_upper_bound,run_num,this_dataset,\
                                                                           plotFlag=True,verbose=True)

# Visualize Fit on Top of Original Cut Data:
# Apply mask of prelimiary quality cuts
mask = (data_df['TotalTileEnergy']>charge_lower_bound) & (data_df['TotalTileEnergy']<charge_upper_bound) \
        & (data_df['TotalSiPMEnergy']>sipm_lower_bound) & (data_df['TotalSiPMEnergy']<sipm_upper_bound) \
        & (data_df['NumTileChannelsHit'] < num_tile_channels_hit) & (data_df['IsFull3D'])

# These coordinates are for a faint point that has labels of the lifetime and quality of fit
mean_x = np.mean(drift_slices)    
mean_y = np.mean(centroids)

# Actually make the Plot
plt.figure(95)
plt.plot(drift_time[mask],charge_energy[mask],'o',color=(0.,0.,1.,0.5),markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.),label='Data')
plt.errorbar(drift_slices,centroids,yerr=centroids_stds,fmt='o',color='black',markersize=5.,\
         markeredgecolor=(0.,0.,0.,0.),label='Alpha Band Fit Centroids (Bins are Left Edges)')
plt.plot(drift_slices,intercept*np.exp(-drift_slices/lifetime),'-',\
         color=(0.,0.,0.,1),linewidth=1.5,markeredgecolor=(0.,0.,0.,0.),\
         label=f'Fit: y = (${intercept:.2f}  \u00B1  ${sigma_intercept:.2f})*exp(-x/(${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f}))')
plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
         label=f'Electron Lifetime: ${lifetime:.2f}  \u00B1  ${sigma_lifetime:.2f} µs')
plt.plot(mean_x,mean_y,'o',color=(0.,0.,0.,0.01),markersize=0.01,markeredgecolor=(0.,0.,0.,0.),\
         label=f'R^2 of Log Fit: {r_squared:.4f}')
plt.xlim([0,np.amax(drift_time[mask])])
plt.ylim([0,1.05*np.amax(charge_energy[mask])])
plt.xlabel("Drift time (µs)")
plt.ylabel("Charge Tile Energy (ADC Counts)")
plt.title("Electron Lifetime, " + run_num + ", "+this_dataset+" ("+str(np.size(centroids))+" Fit Points)")
plt.legend(bbox_to_anchor=(1.1,1.0))

Note that there are some improvements that should be made on this class; it has been committed in its working stage. Please see the class notes for the suggestions for improvement.